# 从零开始AI - 认知服务 - 语音

> ··· 您可以访问 [https://github.com/HaoHoo/F02AI](https://github.com/HaoHoo/F02AI) 来获得 “从零开始AI” 系列全部的内容。··· <br>
>`请以原样使用或转发，欢迎Issue，敬请PR；如果觉得有些意思，欢迎Fork，敬谢Star。` 

从零开始AI的有关介绍中，我们将使用Azure认知服务的标准REST API服务接口，尝试认知服务的一个类别——语音服务。

按照Azure目前提供的语音服务分类，接下来我们可以尝试如下几种能力：
* [文本转语音](#TTS)
* [语音转文本](#STT)

和其他认知服务一样，语音服务也是一种运行在Azure的服务。为了使用Speech API，您需要一个订阅密钥 (subscription key)。别担心，您可以在[这里](https://docs.microsoft.com/azure/cognitive-services/Computer-vision/Vision-API-How-to-Topics/HowToSubscribe)获取免费的订阅密钥。或者在[这里](https://azure.microsoft.com/zh-cn/try/cognitive-services)获取试用的订阅密钥。

获取订阅密钥后，请记录密钥及分配的Azure服务区域。后面我们调用这些API时，必须提供这两种信息。


---
**关于播放声音的说明**

在示例的原始代码中，通过认知服务会生成WAV声音文件，然后需要手动播放。为了能更好地和代码交互，我们会尝试让代码直接播放声音。这部分的代码对jupyter notebook来说，需要安装用于支持声音播放的的库。相对代码简单的播放声音的库可以选择 ‘playsound’。如果是在自己搭建的jupyter notebook环境中运行这个记事本，可以返回到文件列表的界面，在右上角找到‘新建’，然后选择‘终端’。在打开的终端中，使用如下命令安装需要的库：

<code>pip install playsound</code>

'playsound'能够跨平台实现非常简单的声音播放，代码简单到只需要两行
```python
from playsound import playsound
playsound('/path/to/a/sound/file/you/want/to/play.mp3')
```
源码可以在Github上访问 [playsound](https://github.com/TaylorSMarks/playsound)。作者说明了不同系统下工作的机制。
```
On Windows, uses windll.winmm. WAVE and MP3 have been tested and are known to work. Other file formats may work as well.
On OS X, uses AppKit.NSSound. WAVE and MP3 have been tested and are known to work. In general, anything QuickTime can play, playsound should be able to play, for OS X.
On Linux, uses GStreamer. Known to work on Ubuntu 14.04 and ElementaryOS Loki. Support for the block argument is currently not implemented.｜
```
需要说明的是，由于这个库本身存在一些bug，我们可能需要自行做一点修复。

*如果是Windows系统，建议安装之后，打开 playsound.py 找到如下代码并添加差异部分：
```python
if block:
    sleep(float(durationInMS) / 1000.0)
    winCommand('close', alias)
```
*如果是Mac OS系统，运行代码可能会看到需要AppKit之类的报错。安装XCode Command Line Tool并执行：

<code>pip install pyobjc</code>

解决这个问题。如果发现不能播放路径中包含空格的文件，可以打开 playsound.py 找到如下代码并添加差异部分：
```python
        sound = 'file://' + sound
        sound = sound.replace(' ', '%20')
    url   = NSURL.URLWithString_(sound)
```
如果您使用免费的[Azure Notebooks](https://notebooks.azure.com)在线环境运行这个笔记本，由于没有权限运行'pip'安装库文件，您可以按照代码中的提示注释掉或删除用于播放的代码，然后将生成的WAV文件使用其他方式进行播放。

---

## 文字到语音 <a name="TTS"> </a>

准备好了吗？让我们正式开始认知服务从文字到语音的尝试吧。首先需要引入必要的库，然后按照提示输入从免费认知服务试用、免费Azure试用或已有Azure订阅中Speech服务资源里提供的订阅密钥。密钥在我们使用REST API调用时，需要在用于验证的Header中提供，也被称为`Ocp-Apim-Subscription-Key`。

实际上，以下的代码基本来自于微软的示例代码，您可以访问 [Azure Cognitive TTS Samples
](https://docs.microsoft.com/samples/azure-samples/cognitive-speech-tts/azure-cognitive-tts-samples/)来查看简介及下载源代码。为了适合运行在笔记本中，并且直接与认知服务交互，代码做了修改调整。

In [ ]:
import os, requests, time
from xml.etree import ElementTree
#Delete next line if use Azure Notebooks
from playsound import playsound

subscription_key = input('Please input your Service Key:')

您如果还记得之前我们使用认知服务的经历的话，应该对服务区域 `Service Region` 有些印象。有关文本转语音服务的服务区域，可以查看对应的 [Speech-to-text REST API](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/rest-speech-to-text) 文档中服务区域的介绍。服务区域也就是提供认知服务的区域，一般我们获得的订阅密钥，是和服务区域关联的。
因此，需要在查看服务密钥的地方，确认对应的服务区域识别字符串，并按照以下代码提示输入。

In [ ]:
service_region = input('please input your Service Region:')

对于基于语音的认知服务，由于很多时候对声音的处理是相对异步的，所以每个服务调用的会话，需要生成一个`token`来进行验证和返回。获取这个`token`的方法非常简单，只需要使用POST方法向认知服务生成`token`的服务终结点URL提交一个含有订阅密钥头部header的请求即可。返回的字符串就是我们后续调用服务需要的`token`。为了看看`token`到底长什么样，我们可以将其显示出来。

In [ ]:
fetch_token_url = "https://"+service_region+".api.cognitive.microsoft.com/sts/v1.0/issueToken"
headers = {
            'Ocp-Apim-Subscription-Key': subscription_key
        }
response = requests.post(fetch_token_url, headers=headers)
access_token = str(response.text)

print(access_token)

拿到了`token`，我们就可以尝试调用语音认知服务了。对于文本转语音的服务，有专门的服务终结点URL，我们需要按照REST API的要求，构造这个服务终结点的URL并发起REST API调用。

与之前我们尝试过的计算机视觉等认知服务不同，提交一个文本转语音的请求时，并不是直接使用订阅密钥，而是使用由其生成的`token`。用法是在头部header中包含一个`Authorization`字段，内容是以`Bearer `开头并加上`token`的字段。

让我们先尝试一个简单的服务调用：拿到支持的语音清单。通过这个API，我们可以得到在当前服务区域可供使用的各种语言以及语言对应的语音模型。

In [ ]:
base_url = "https://"+service_region+".tts.speech.microsoft.com/"
path = 'cognitiveservices/voices/list'
constructed_url = base_url + path
headers = {
            'Authorization': 'Bearer ' + access_token,
        }
response = requests.get(constructed_url, headers=headers)
if response.status_code == 200:
    print("\nAvailable voices: \n" + response.text)
else:
    print("\nStatus code: " + str(response.status_code) +  \
          "\nSomething went wrong. Check your subscription key and headers.\n")


返回的是标准的JSON格式数据。我们可以找一下，对于简体中文 `zh-CN`，认知服务提供了慧慧 `Huihui`、康康 `Kangkang`和瑶瑶 `Yaoyao`。这几个标准语音有性别区分。注意看有个数据字段叫做 `ShortName`，后面我们选择文本转语音使用的标准语音类型，就可以通过这个名称来选择。除了标准语音，在部分区域例如`westus2`等，还支持神经语音标准，能够提供更加自然顺畅的语音，并且可以使用[语音合成标记语言](https://docs.microsoft.com/zh-cn/azure/cognitive-services/speech-service/speech-synthesis-markup#adjust-speaking-styles)来配置和调整神经语音。

语音服务能支持的语言范围，可以在[语音服务的语言和语音支持](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/language-support#speech-to-text)这一文档中查看。

返回的数据中还有一个参数`SampleRateHertz`采样率，这与认知服务支持的音频格式有关，我们将在POST代码部分时候介绍支持的音频格式。

接下来，我们开始为文本转语音做些准备工作。首先做一个时间戳，提供给WAV文件生成时用来产生文件名。然后，我们需要按照提示，输入用于转变成语音的文本。

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M")
ttsstr = input("What would you like to convert to speech: ")

首先我们生成一个用于给WAV文件命名的时间戳。

然后，我们按照提示输入需要进行语音转换的文本。

In [ ]:
base_url = "https://"+service_region+".tts.speech.microsoft.com/"
path = 'cognitiveservices/v1'
constructed_url = base_url + path
headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/ssml+xml',
        'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
        'User-Agent': 'YOUR_RESOURCE_NAME'
        }

正式POST一个文本转语音的请求时，API的服务终结点需要指定为特定的路径。同时，头部header除了之前使用的`token`之外，还需要说明后续`bady`的内容类型，以及需要返回的声音输出格式。

语音认知服务目前能够支持的声音类型如下：

```output
raw-16khz-16bit-mono-pcm            raw-8khz-8bit-mono-mulaw
riff-8khz-8bit-mono-alaw            riff-8khz-8bit-mono-mulaw
riff-16khz-16bit-mono-pcm           audio-16khz-128kbitrate-mono-mp3
audio-16khz-64kbitrate-mono-mp3     audio-16khz-32kbitrate-mono-mp3
raw-24khz-16bit-mono-pcm            riff-24khz-16bit-mono-pcm
audio-24khz-160kbitrate-mono-mp3    audio-24khz-96kbitrate-mono-mp3
audio-24khz-48kbitrate-mono-mp3     ogg-24khz-16bit-mono-opus
```

此外，提交请求的`bady`部分需要满足[Speech Synthesis Markup Language (SSML)](https://docs.microsoft.com/en-us/azure/cognitive-services/speech-service/speech-synthesis-markup)的格式要求。

In [ ]:
xml_body = ElementTree.Element('speak', version='1.0')
# xml_body.set('{http://www.w3.org/XML/1998/namespace}lang', 'en-us')
xml_body.set('{http://www.w3.org/XML/1998/namespace}lang', 'zh-cn')
voice = ElementTree.SubElement(xml_body, 'voice')
# voice.set('{http://www.w3.org/XML/1998/namespace}lang', 'en-US')
voice.set('{http://www.w3.org/XML/1998/namespace}lang', 'zh-cn')
# voice.set('name', 'en-US-Guy24kRUS') 
# Short name for 'Microsoft Server Speech Text to Speech Voice (en-US, Guy24KRUS)'
voice.set('name', 'zh-CN-HuihuiRUS') 
# Short name for 'Microsoft Server Speech Text to Speech Voice (zh-CN, HuihuiRUS)'
voice.text = ttsstr
body = ElementTree.tostring(xml_body)

提供一个SSML格式`body`，有两部分。

* 首先需要按照 w3.org 的规范，给出文档的描述。

```XML
<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="string"></speak>
```

* 然后需要对具体使用的语音进行描述。在上述代码示例中，我们选择使用简体中文，并按照认知语音服务对于区域所支持的语音，选择标准语音的慧慧的声音`zh-CN-HuihuiRUS`。

```XML
    <voice name="zh-CN-HuihuiRUS"></voice>
```

* 最后，我们传递之前输入的文本到这个XML文件中。最终的文档就形如：

```XML
<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="string">
    <voice name="zh-CN-HuihuiRUS">
        认识服务是简单而神奇的人工智能体现。
    </voice>
</speak>
```

接下来的工作，就是把构造好的头部headers、包含SSML的bady提交到REST API的服务终结点地址了。

In [ ]:
response = requests.post(constructed_url, headers=headers, data=body)
if response.status_code == 200:
    with open('sample-' + timestr + '.wav', 'wb') as audio:
        audio.write(response.content)
        print("\nStatus code: " + str(response.status_code) + "\nYour TTS is ready for playback.\n")
#Delete next line if use Azure Notebooks
        playsound('sample-' + timestr + '.wav')
else:
    print("\nStatus code: " + str(response.status_code) + "\nSomething went wrong. Check your subscription key and headers.\n")
    print("Reason: " + str(response.reason) + "\n")

假如一切顺利，上述代码运行后，就能听到由您输入的文本生成的语音了。提交请求后，语音认知服务会返回声音的内容，将返回的数据写入一个WAV文件，就可以进行播放了。

当jupyter notebook能够使用终端运行`pip`导入库时，就可以通过`playsound`直接播放这个文件了。

播放结束后，可使用以下代码输入 y 或 n 来决定是否删除生成的WAV文件。

In [ ]:
delfile = input("Delete the WAV file? (y/n):")
if delfile=='y':
    os.remove('sample-' + timestr + '.wav')

如果您选择了支持神经语音的服务区域，例如`westus2`，强烈建议您返回到构造SSML的代码块，将“慧慧”的标准语音对应的"zh-CN-HuihuiRUS"改为“晓悠“的神经语音"zh-CN-XiaoyouNeural"，然后好好的比较两者之间的差异。

---
**关于录制声音的说明**

前面我们使用的`playsound`用于简单的播放声音，但如果我们想通过麦克风录制声音然后让语音认知服务进行识别的时候，就需要一个支持麦克风输入的库了。所以我们来安装一个新的库:

<code>pip install pyaudio</code>

*在macOS上运行pip安装的时候，会报错提示找不到 portaudio.h 。解决这个问题并不复杂，使用如下 brew 命令行安装 portaudio 即可：

<code>brew install portaudio</code>

*在macOS Catalina上运行代码录音时，您也许会发现录制的WAV文件没有任何声音。估计这是由于系统偏好设置中，关于隐私的配置的问题。如果终端没有提示要访问麦克风，加入到麦克风的允许访问列表，运行代码时就不能访问麦克风。一个可以参考的做法是，安装一个命令行下访问麦克风的软件包，然后尝试触发终端的麦克风权限请求。

<code>brew install sox</code>

安装之后，就可以直接运行 sox 尝试让终端访问麦克风。还可以检查这个命令行生成的 test.wav 文件是否已经成功地录入了声音。

<code>sox -d test.wav</code>

按照网上的建议，还可以手动打开一个终端再运行上述步骤：

<code>open /System/Applications/Utilities/Terminal.app/Contents/MacOS/Terminal</code>

如果已经能够通过终端录制声音，我们就可以进入正式的代码尝试了。

---

## 语音到文字 <a name="STT"> </a>

关于语音到文字的智能识别，在微软的示例代码中却没找到 Python 的，所以参考[Speech-to-Text REST API](https://docs.microsoft.com/zh-cn/azure/cognitive-services/speech-service/rest-speech-to-text)文档中 C# 的示例代码，写出了以下 Python 的实现。

实际上，如果我们顺着代码块一直运行到这里，是不需要再重新运行代码输入服务订阅密钥和服务区域的。为了避免使用超链接直接跳转到这里运行代码，缺少服务订阅密钥和服务区域导致报错，我们又一次使用代码提供了这些信息。

In [ ]:
import os, requests
subscription_key = input('Please input your Service Key:')

In [ ]:
service_region = input('please input your Service Region:')

fetch_token_url = "https://"+service_region+".api.cognitive.microsoft.com/sts/v1.0/issueToken"
headers = {
            'Ocp-Apim-Subscription-Key': subscription_key
        }
response = requests.post(fetch_token_url, headers=headers)
access_token = str(response.text)

print(access_token)

为了尽可能简化代码，我们可以首先用 `pyaudio` 来按照录制一个WAV文件，[Speech-to-Text REST API](https://docs.microsoft.com/zh-cn/azure/cognitive-services/speech-service/rest-speech-to-text)文档中介绍，通过REST API调用时，语音认知服务接受两种文件编码格式：

| Format | Codec |Bit rate | Sample Rate |
| - | - | - | - |
| WAV | PCM | 256 kbps | 16 kHz, mono |
| OGG | OPUS | 256 kpbs | 16 kHz, mono|

因此，我们使用 `pyaudio` 录制语音以及保存为WAV文件时，应该按照这个标准设置好参数。录制将使用16 kHz的采样率、16 bit的比特率，单声道。为了减少等待时间，我们将录制的时间定义为 5 秒钟。如果需要，可以自行修改这个参数。
我们使用一个名为 'test.wav' 的文件保存录制的语音。

In [ ]:
import pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "test.wav"

为了在包含多个支持录音的音频设备的计算机上运行这些代码，我们使用 `pyaudio` 来枚举这些设备，并让您选择使用哪一个。

当看到 'Recording...' 的提示时，表示录音已经开始。您可以说一句简单的英文（因为后面的代码我们选择的语言是 `en-US`，如果您录制的是中文，请修改后续的代码）。大约 5 秒钟左右，'End record.' 会显示，表示录音结束。当然，录音时长可在前面的代码处修改。

录制结束后，自带的 `wave` 库就会将生成的流写为一个WAV文件，供后续使用。 

In [ ]:
audio = pyaudio.PyAudio()

print("----------------------record device list---------------------")
info = audio.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
for i in range(0, numdevices):
    if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
        print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))

print("-------------------------------------------------------------")

index = int(input("Please select device id:"))
print("recording via index "+str(index))
stream = audio.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("Recording...")
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print("End record.")

stream.stop_stream()
stream.close()
audio.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(audio.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

接下来就是正式的语音认知服务的尝试了。我们会按照[Speech-to-Text REST API](https://docs.microsoft.com/zh-cn/azure/cognitive-services/speech-service/rest-speech-to-text)文档的要求，提交一个语音到文本的认知服务请求给 REST API。

首先需要提供的是服务的终结点URL，形如：
<code>https://&lt;REGION_IDENTIFIER&gt;.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1</code>

我们将使用 `base_url` 加上 `path` 构造出完整的服务终结点URL。

    ⚠️ 需要注意的是，必须将语言参数追加到 URL 以避免收到 4xx HTTP 错误。

接着我们提供必要的参数给POST方法。重要的参数主要有这几个：
    
| 参数 | 说明 | 必需/可选 |
| :-: | :- | :-: |
| language | 标识所要识别的口语。 请参阅支持的语言。 | 必须 |
| format | 指定结果格式。 接受的值为 simple 和 detailed。 简单结果包括 RecognitionStatus、DisplayText、Offset 和 Duration。 Detailed 响应包括显示文本的四种不同的表示形式。 默认设置为 simple。 | 可选 |

一个标准的HTTP的POST方法还需要提供头部headers的参数。以下是主要的参数：

| 标头 | 说明 | 必需/可选 |
| :-: | :- | :- |
| Ocp-Apim-Subscription-Key | 语音服务订阅密钥。 | 此标头或 Authorization 是必需的。|
| Authorization | 前面带有单词 Bearer 的授权令牌。 有关详细信息，请参阅身份验证。 | 此标头或 Ocp-Apim-Subscription-Key 是必需的。|
| Content-type | 描述所提供音频数据的格式和编解码器。 接受的值为 audio/wav; codecs=audio/pcm; samplerate=16000 和 audio/ogg; codecs=opus。| 必需 |
| Accept | 如果提供此标头，则值必须是 application/json。 语音服务以 JSON 格式提供结果。 某些请求框架提供不兼容的默认值。 最好始终包含 Accept。| 可选，但建议提供。|

与文本到语音不同，除了 token，语音到文本也支持订阅密钥的直接验证。两者二选一即可。
    
最后是POST这个请求最重要的部分，我们将打开之前录制的WAV文件，将其作为请求的body使用。并且对返回的响应进行阅读和分析。

In [ ]:
base_url = "https://"+service_region+".stt.speech.microsoft.com/"
path = 'speech/recognition/conversation/cognitiveservices/v1'
constructed_url = base_url + path
params = {
        'language': 'en-US',
        'format': 'detailed'
        }
headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'audio/wav; codecs=audio/pcm; samplerate=16000',
        'Accept': 'application/json;text/xml'
        }
body = open('test.wav','rb').read()
response = requests.post(constructed_url, params=params, headers=headers, data=body)
if response.status_code == 200:
    print(response.json())
else:
    print("\nStatus code: " + str(response.status_code) + "\nSomething went wrong. Check your subscription key and headers.\n")
    print("Reason: " + str(response.reason) + "\n")

如果一切正常，语音认知服务将返回JSON格式的数据。 如果前面参数我们选择的是 'simple' ，就能看到以下数据：

| 参数 | 说明 |
| :-: | :- |
| RecognitionStatus | 状态，例如 Success 表示成功识别。|
| DisplayText | 经过大小写转换、添加标点、执行反向文本规范化（将口头文本转换为短形式，例如，200 表示“two hundred”，或“Dr.Smith”表示“doctor smith”）和屏蔽亵渎内容之后的识别文本。 仅在成功时提供。|

如果像上面代码中，选择了 'detailed' 参数，就能看到：

| 参数 | 说明|
| :-: | :- |
| Confidence | 条目的置信度评分，从 0.0（完全不可信）到 1.0（完全可信）|
| Lexical | 已识别文本的词法形式：识别的实际单词。|
| ITN | 已识别文本的反向文本规范化（“规范”）形式，已应用电话号码、数字、缩写（“doctor smith”缩写为“dr smith”）和其他转换。|
| MaskedITN | 可根据请求提供应用了亵渎内容屏蔽的 ITN 形式。|
| Display | 已识别文本的显示形式，其中添加了标点符号和大小写形式。 此参数与将格式设置为 simple 时提供的 DisplayText 相同。|

您在返回的数据中，找到了语音认知服务识别出的文本了吗？

同样，运行完毕后，您可以输入 y 或 n 来决定是否删除使用过的 test.wav 文件。

In [ ]:
delfile = input("Delete the WAV file? (y/n):")
if delfile=='y':
    os.remove('test.wav')